- 결과 데이터가 0과 1로 구성되어 있다
- 2진 분류
- 손실함수 : binary_crossentropy
- 출력층의 활성화 함수 : sigmoid
- 출력층의 노드의 개수 : 1

In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [2]:
# 데이터를 읽어온다.
df1 = pd.read_csv('./data/pima-indians-diabetes.csv', header=None)
df1.columns = ['임신횟수', '혈당', '혈압', '근육피부두께', '인슐린', '체질량', '가족력', '나이', '당뇨여부']
df1.head()

,임신횟수,혈당,혈압,근육피부두께,인슐린,체질량,가족력,나이,당뇨여부
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   임신횟수    768 non-null    int64  
 1   혈당      768 non-null    int64  
 2   혈압      768 non-null    int64  
 3   근육피부두께  768 non-null    int64  
 4   인슐린     768 non-null    int64  
 5   체질량     768 non-null    float64
 6   가족력     768 non-null    float64
 7   나이      768 non-null    int64  
 8   당뇨여부    768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
# 결측치 확인
df1.isna().sum()

임신횟수      0
혈당        0
혈압        0
근육피부두께    0
인슐린       0
체질량       0
가족력       0
나이        0
당뇨여부      0
dtype: int64

In [5]:
# 입력과 결과로 나눈다.
X = df1.drop('당뇨여부', axis=1)
y = df1['당뇨여부']

In [6]:
# 2진 분류 옵션들
# 출력층의 노드의 개수
out_nodes = 1
# 손실함수
loss_function = 'binary_crossentropy'
# 활성화함수
activation_function = 'sigmoid'

In [7]:
# 표준화
scaler1 = StandardScaler()
X = scaler1.fit_transform(X)
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [8]:
# 입력노드의 개수
# 입력 데이터 행 하나의 컬럼의 개수
input_size = X.shape[1]
input_size

8

In [9]:
# 신경망 설계
model = Sequential()

model.add(Dense(12, input_dim=input_size))
model.add(Activation('relu'))

# model.add(Dense(8))
# model.add(Activation('relu'))

# model.add(Dense(6))
# model.add(Activation('relu'))

# model.add(Dense(4))
# model.add(Activation('relu'))

# model.add(Dense(2))
# model.add(Activation('relu'))

model.add(Dense(out_nodes))
model.add(Activation(activation_function))

In [10]:
# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                108       
                                                                 
 activation (Activation)     (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 activation_1 (Activation)   (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 6)                 54        
                                                                 
 activation_2 (Activation)   (None, 6)                 0         
                                                                 
 dense_3 (Dense)             (None, 4)                 2

In [11]:
# 학습
model.fit(X, y, epochs=200, batch_size=10)

Epoch 1/200
77/77 [==============================] - 1s 3ms/step - loss: 0.6894 - accuracy: 0.6510
Epoch 2/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6825 - accuracy: 0.6510
Epoch 3/200
77/77 [==============================] - 0s 4ms/step - loss: 0.6765 - accuracy: 0.6510
Epoch 4/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6714 - accuracy: 0.6510
Epoch 5/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6672 - accuracy: 0.6510
Epoch 6/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6636 - accuracy: 0.6510
Epoch 7/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6607 - accuracy: 0.6510
Epoch 8/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6581 - accuracy: 0.6510
Epoch 9/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6561 - accuracy: 0.6510
Epoch 10/200
77/77 [==============================] - 0s 3ms/step - loss: 0.6543 - accuracy: 0.6510
Epoch 11/

In [12]:
# 정확도 확인
a1 = model.evaluate(X, y)
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')

24/24 [==============================] - 0s 2ms/step - loss: 0.6468 - accuracy: 0.6510
손실률 : 0.6467998623847961
정확도 : 0.6510416865348816
